In [1]:
!pip install dtaidistance

     |████████████████████████████████| 797 kB 26.6 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dtaidistance: filename=dtaidistance-2.3.6-cp36-cp36m-linux_x86_64.whl size=888971 sha256=26392f60c45fc968ef776a2bd8dd1ccc3d264bd48acfded796ca09e989f00dfe
  Stored in directory: /home/ec2-user/.cache/pip/wheels/30/5f/18/3db8e0572e06cde30543380d60b4511e3802d1481c3a3a2642
Successfully built dtaidistance


In [2]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from collections import defaultdict
from tqdm import tqdm 
from dtaidistance import dtw

tqdm.pandas()

In [4]:
df_pred = pd.read_csv("../Data/df_pred.csv").drop("Unnamed: 0", axis=1)

In [5]:
df_pred.head()

,candidate_id,job_order,time_between,time_spent,isco_functie_niveau,education,function_id,isco_code4
0,84556,0,37,156,2.0,0.0,937.0,207.0
1,84556,1,23,116,1.0,0.0,811.0,347.0
2,84556,2,23,275,2.0,0.0,937.0,207.0
3,84556,3,1155,4,1.0,0.0,1521.0,343.0
4,84556,4,203,11,1.0,0.0,1521.0,343.0


In [6]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [7]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 465468/465468 [00:11<00:00, 38994.82it/s]


In [8]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [ ]:
results = defaultdict(lambda: defaultdict(int))

try:
    for i, current in tqdm(enumerate(career_paths.iteritems()), total=len(career_paths)):
        candidate, career = current
        career = career[:-1]

        for j in career_paths.iteritems():
            candidate2, career2 = j

            if candidate != candidate2:
                results[candidate][candidate2] = dtw.distance(career, 
                                                              career2, 
                                                              use_c=True, 
                                                              use_pruning=True)
                
        if i == 2000:
            break
                        
except KeyboardInterrupt:
    pass

  1%|          | 1569/265647 [58:08<407:27:08,  5.55s/it]

In [ ]:
best_matches = {}

for i in tqdm(results):
    if len(results[i]):
        best_matches[i] = sorted(results[i], key=lambda x: results[i][x])

In [ ]:
def knn_classifier(best_matches, career_paths, k=1):
    
    predictions = []
    
    for candidate in best_matches.keys():
        correct = career_paths[candidate][-1]
                
        neighbors = [(career_paths[neighbor][-1], i + 1) for i, neighbor in enumerate(best_matches[candidate][:k])]
        
        weights = defaultdict(int)
        
        for job, weight in neighbors:
            weights[job] += 1/np.log(1 + weight)
            
        predictions.append(correct == max(weights, key=lambda x: weights[x]))
        
    return np.mean(predictions)

In [ ]:
results = defaultdict(list)

for k in tqdm(range(1, 20)):
    results["k"].append(k)
    results["accuracy"].append(knn_classifier(best_matches, career_paths, k=k))
    
pd.DataFrame(results).set_index("k")